## 네이버 영화평 감성 분석 - TfidfVectorizer

In [1]:
import pandas as pd
import numpy as np

In [33]:
#전처리끝난 파일 불러옴
train_df = pd.read_csv('../00.data/NaverMovie/train.tsv', sep='\t')
test_df = pd.read_csv('../00.data/NaverMovie/test.tsv', sep='\t')

In [21]:
train_df.head(3)

,Unnamed: 0,id,document,label
0,0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,2,10265843,너무재밓었다그래서보는것을추천한다,0


아까 04_는 토큰화해서 넣어줬는데, 토크나이저함수를 써서 해볼게용  
은는이가을를 이런것들 확률을 낮춰줄꺼니까
### Tokenizer 함수 정의

In [9]:
from konlpy.tag import Okt

okt = Okt()
def tw_tokenizer(text):
    tokens_ko = okt.morphs(text)
    return tokens_ko

### TfidfVectorizer로 학습/변환

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvecter = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9) #여기에 함수사용

In [11]:
%time tvecter.fit(train_df.document) #%time 소요시간안내해줌
# 교수님코드: %time tvecter.fit(train_df['document'])
#15분돌아갈지도...8분소요됨
#결과값 Wall time: 8min 18s
# TfidfVectorizer(max_df=0.9, min_df=3, ngram_range=(1, 2),
#                 tokenizer=<function tw_tokenizer at 0x000001D25806AB80>)

Wall time: 8min 18s


TfidfVectorizer(max_df=0.9, min_df=3, ngram_range=(1, 2),
                tokenizer=<function tw_tokenizer at 0x000001D25806AB80>)

In [12]:
# X_train만들기 12:02~12:11 9분소요
%time X_train_tvect = tvecter.transform(train_df['document'])

In [13]:
# X_test만들기 12:11~12:15 4분소요
%time X_test_tvect = tvecter.transform(test_df['document'])

Wall time: 2min 49s


In [35]:
y_train = train_df.label.values
y_test = test_df.label.values

### LogisticRegression으로 학습/예측/평가

In [14]:
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()

In [15]:
from sklearn.metrics import accuracy_score
lr_clf.fit(X_train_tvect, train_df.label)
pred = lr_clf.predict(X_test_tvect)
accuracy_score(test_df.label, pred)
#그냥 했을때 결과값:0.8531

0.8531074599448923

In [36]:
#교수님코드:LogisticRegression으로 학습/예측/평가 
lr_clf = LogisticRegression(C=3.5)
lr_clf.fit(X_train_tvect, y_train)
pred = lr_clf.predict(X_test_tvect)
accuracy_score(y_test, pred)
#교수님결과 0.8584

0.8584753546280233

### 실제테스트 04_에 이어서 또 해보기;; 토크나이저함수가 있어서 별도의 형태소분석을 해줄 필요가 없다 

In [18]:
review1 = '진짜 개노잼이다.. 1편이랑 같은 감독맞나?러닝타임도 길어서 개지루함 ㄹㅇ'
review2 = '이런 사랑영화가 다시 나올 수 있을까?'

In [22]:
import re
review = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", review1)
review_tvect = tvecter.transform([review])
pred = lr_clf.predict(review_tvect)
pred[0] #0부정 1긍정 :결과값1

0

In [23]:
review = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", review2) #리뷰2를 넣었더니
review_tvect = tvecter.transform([review])
pred = lr_clf.predict(review_tvect)
pred[0] #0부정 1긍정 :결과값1 긍정으로 평가를 잘했다.

1

In [24]:
reviews = ['진짜 개노잼이다.. 1편이랑 같은 감독맞나?러닝타임도 길어서 개지루함 ㄹㅇ', '이런 사랑영화가 다시 나올 수 있을까?']

In [30]:
reviews = list(map(lambda x: re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", x), reviews))
print(reviews)

['진짜 개노잼이다 편이랑 같은 감독맞나러닝타임도 길어서 개지루함 ㄹㅇ', '이런 사랑영화가 다시 나올 수 있을까']


In [31]:
review_tvect = tvecter.transform(reviews)
pred = lr_clf.predict(review_tvect)
pred[0], pred[1]

(0, 1)

### GridSearchCV로 최적의 파라미터 찾기

In [37]:
from sklearn.model_selection import GridSearchCV

In [38]:
lr_clf = LogisticRegression()
params = {
    'C': [1, 3, 3.5, 5]
}

grid_cv = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1) #n_jobs=-1
grid_cv.fit(X_train_tvect, y_train)
print(grid_cv.best_params_, grid_cv.best_score_)
#2분소요
# 결과값: Fitting 3 folds for each of 4 candidates, totalling 12 fits
# [Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
# [Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:   50.7s finished
# {'C': 3.5} 0.8550664993037979

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:   50.7s finished
{'C': 3.5} 0.8550664993037979


In [39]:
pred = grid_cv.predict(X_test_tvect)
acc = accuracy_score(y_test, pred)
print(f'TfidfVectorizer + Logistic Regression 정확도: {acc:.4f}')
#결과값 0.8585

TfidfVectorizer + Logistic Regression 정확도: 0.8585
